# Clustering on term frequencies

In [ ]:
import pandas as pd
from sklearn import 

http://scikit-learn.org/stable/modules/clustering.html#k-means
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score

In [ ]:
X = pd.concat(
    (
        #pd.read_csv('../features/metadata/tf_descriptions.csv', index_col=0),
        pd.read_csv('../features/metadata/tf_keywords.csv', index_col=0),
        pd.read_csv('../features/metadata/tf_titles.csv', index_col=0),
    ),
    axis=1
)